In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import text


In [ ]:
HOST = "172.18.0.7"
PORT = "5435"
USER = "moodle_sink"
PASS = "psqlpassword"
DB = "sink"
CONNECTION_STR = f"postgresql+psycopg2://{USER}:{PASS}@{HOST}/{DB}"
ENGINE = create_engine(CONNECTION_STR)




In [ ]:
with ENGINE.connect() as conn:
    select_sites_users = 'SELECT * FROM "SitesUsers";'
    sites_users = pd.read_sql(sql=select_sites_users, con=conn)#, parse_dates=['registration_timestamp'])
    select_community_users = 'SELECT * FROM "CommunityUsers";'
    community_users = pd.read_sql(sql=select_community_users, con=conn)#, parse_dates=['registration_timestamp'])

In [ ]:
sites_users['registration_timestamp'] = pd.to_datetime(sites_users['registration_timestamp'], unit="s")
community_users['registration_timestamp'] = pd.to_datetime(community_users['registration_timestamp'], unit="s")


# Data info

In [ ]:
sites_users.info()

In [ ]:
community_users.info()

In [ ]:
sites_users.head()

In [ ]:
sites_users['year'] = sites_users['registration_timestamp'].dt.year
community_users['year'] = community_users['registration_timestamp'].dt.year

In [ ]:
sites_users.groupby("year")["admin_email"].count().plot(kind="bar", title="Site Users Count Per Year", ylabel="Users Count")


In [ ]:
community_users.groupby("year")["email"].count().plot(kind="bar", title="Community Users Count Per Year", ylabel="Users Count")


In [ ]:
sites_users['email'] = sites_users['admin_email']

In [ ]:
fourm_member_admins = sites_users[sites_users['admin_email'].isin(community_users['email'])]

In [ ]:
fourm_member_admins.shape

In [ ]:
fourm_member_admins

In [ ]:
fourm_member_admins.groupby("year")["admin_email"].count().plot(kind="bar", title="Site Admins that are also community users", ylabel="count")

# user count by week of year

In [ ]:
sites_users["week_of_year"] = sites_users['registration_timestamp'].dt.isocalendar().week